# AE 771 | Homework \#11 | Rocket Propulsion | Ray Taghavi
### Code Written by Henry Hunt

In [8]:
#Importing Libraries
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sympy import Symbol
import time
import math

#Graph Formatting
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 14
fig_size[1] = 10
plt.rcParams["figure.figsize"] = fig_size

#Graph Font Edit
font = {'family':'Times New Roman',
        'weight' : 'bold',
        'size'   : 22}
matplotlib.rc('font', **font)

In [9]:
#Question 1
#Part A | Reproduce Figure 4.6



#Part B | Calculate the orbital velocity and escape velocity at the distances above Earth
d1 = 200  #Distance above Earth 1 | miles
d2 = 2150 #Distance above Earth 2 | miles

#Part C | Calcualte the orbit altitude for geostationary and geo-synchronous satellites
re = 4000 #Mean Radius of the Earth | miles



In [28]:
#Question 2
#Part A | Determine velocity increment achievable for single stage system
mi = 25000 #Initial mass | lbm
ms = 2000  #Structural mass | lbm
mp = 500   #Payload mass | lbm
Isp = 300  #Specific impulse | sec

#Single Stage Change in Velocity Equations
    #c = 0
ζ = 0.95 #0.99 for multi
MR = 0.05 #0.01 for multi
    #go = 0
    #l = 0 #Payload Fraction
    #Δu = -c*ln(1-ζ)
    #Δu = -c*ln(MR)
    #Δu = c*ln(1/MR)
    #Δu = Isp*go*ln(1/MR)
    
#Calculation Process
s = ms/mi                  #Structural Fraction
l = mp/mi                  #Payload Fraction
go = 32.2                  #Gravitational Constant | ft/s^2
Δu = Isp*go*np.log(1/(s+l))#Velocity increment | ft/s
print('The achievable velocity increment is',round(Δu,2),'ft/s')

#Part B | If the vehicle reaches an altitude of 200 miles can it obtain orbital velocity?


The achievable velocity increment is 22242.97 ft/s


In [22]:
#Question 3
# Two Stage booster at Kennedy Space Center
vdes = 10000 #Deisgn Velocity of Booster | m/s
ms2 = 12000  #Mass of the second stage including structure and propellant | kg
ms2p = 9000  #Mass of just the stage 2 propellant | kg
mp = 2000    #Mass of the payload | kg
Isp1 = 350   #Isp of the first stage | sec
Isp2 = 400   #Isp of the second stage |sec
go =9.81     #Gravity | m/sec^2
ms1 = 8000   #Mass of stage 1 structural components | kg
#What mass of propellant must be loaded on the first stage to achieve the required design velocity?
l1 = mp/ms1
l2 = 0 #Payload Fraction
s2 = (ms2-ms2p)/ms2 #Structural Fraction
#Δu1 = Isp*go*ln(1/(s1+l1))
Δu2 = Isp2*go*np.log(1/(s2+l2))
Δu1 = vdes - Δu2
ms1f = ((np.exp((Δu1/Isp1)/go))**(-1)-l1)*ms1  #Mass of the stage 1 fuel | kg
print(Δu2)
print("The mass of the first stage propellant is",round(ms1f,2),"kg.")


#What is the vehicles total mass at lift-off?
mtot = (ms2 + mp + ms1 + ms1f)
print("The total mass at lift-off of the vehicle is",round(mtot,2),"kg.")

5439.81907303445
The mass of the first stage propellant is 119.75 kg.
The total mass at lift-off of the vehicle is 22119.75 kg.
